In [1]:
import geopy 
from geopy.geocoders import Bing
from geopy.geocoders import Nominatim
import geopandas
import pandas as pd
from time import sleep
import requests
from requests.auth import HTTPBasicAuth

In [51]:
from multiprocessing import Pool
n_processes=20

In [36]:
#Read in the patient addresses
patients = pd.read_csv('patient_addresses.csv', encoding='latin-1')

#Read in the care center addresses
care_centers = pd.read_csv('ng_locations.csv', encoding='latin-1')


In [49]:
#Take a look at the dataset
address_sample = patients.sample(10)
address_sample

,Patient Address,Patient Address Line 1,Patient City,Patient ZIP,Patient State
317598,"15403 40th Drive SE Bothell, WA 98012",15403 40th Drive SE,Bothell,98012,WA
184505,"5124 322nd Ave Se Fall City, WA 98024",5124 322nd Ave Se,Fall City,98024,WA
23978,"350 Chrisman Oaks Trl Nicholasville, KY 40356",350 Chrisman Oaks Trl,Nicholasville,40356,KY
273629,"11610 Hunter Lane NW Gig Harbor, WA 98332",11610 Hunter Lane NW,Gig Harbor,98332,WA
223024,"7605 Snowberry Ave SE Snoqualmie, WA 98065",7605 Snowberry Ave SE,Snoqualmie,98065,WA
306206,"14322 Holly Burn Ln NW Gig Harbor, WA 98329",14322 Holly Burn Ln NW,Gig Harbor,98329,WA
343132,"18016 29th Dr SE Mill Creek, WA 98012",18016 29th Dr SE,Mill Creek,98012,WA
141400,"3375 Morningstar Dr Bellingham, WA 98226",3375 Morningstar Dr,Bellingham,98226,WA
424658,"Po Box 1801 Buckley, WA 98321",Po Box 1801,Buckley,98321,WA
293877,"13241 SE 43rd Street Bellevue, WA 98006",13241 SE 43rd Street,Bellevue,98006,WA


In [50]:
#Convert the above sample addresses so that the API can read them in
address_sample['converted_address'] = address_sample['Patient Address'].apply(lambda x: x.replace(' ', '%20'))
address_sample

,Patient Address,Patient Address Line 1,Patient City,Patient ZIP,Patient State,converted_address
317598,"15403 40th Drive SE Bothell, WA 98012",15403 40th Drive SE,Bothell,98012,WA,"15403%2040th%20Drive%20SE%20Bothell,%20WA%2098012"
184505,"5124 322nd Ave Se Fall City, WA 98024",5124 322nd Ave Se,Fall City,98024,WA,"5124%20322nd%20Ave%20Se%20Fall%20City,%20WA%20..."
23978,"350 Chrisman Oaks Trl Nicholasville, KY 40356",350 Chrisman Oaks Trl,Nicholasville,40356,KY,"350%20Chrisman%20Oaks%20Trl%20Nicholasville,%2..."
273629,"11610 Hunter Lane NW Gig Harbor, WA 98332",11610 Hunter Lane NW,Gig Harbor,98332,WA,"11610%20Hunter%20Lane%20NW%20Gig%20Harbor,%20W..."
223024,"7605 Snowberry Ave SE Snoqualmie, WA 98065",7605 Snowberry Ave SE,Snoqualmie,98065,WA,"7605%20Snowberry%20Ave%20SE%20Snoqualmie,%20WA..."
306206,"14322 Holly Burn Ln NW Gig Harbor, WA 98329",14322 Holly Burn Ln NW,Gig Harbor,98329,WA,"14322%20Holly%20Burn%20Ln%20NW%20Gig%20Harbor,..."
343132,"18016 29th Dr SE Mill Creek, WA 98012",18016 29th Dr SE,Mill Creek,98012,WA,"18016%20%2029th%20Dr%20SE%20Mill%20Creek,%20WA..."
141400,"3375 Morningstar Dr Bellingham, WA 98226",3375 Morningstar Dr,Bellingham,98226,WA,"3375%20Morningstar%20Dr%20Bellingham,%20WA%209..."
424658,"Po Box 1801 Buckley, WA 98321",Po Box 1801,Buckley,98321,WA,"Po%20Box%201801%20Buckley,%20WA%2098321"
293877,"13241 SE 43rd Street Bellevue, WA 98006",13241 SE 43rd Street,Bellevue,98006,WA,"13241%20SE%2043rd%20Street%20Bellevue,%20WA%20..."


In [42]:
#Get lat, lons for each address in the converted_address column

def get_lat_lon(address):
    #Our endpoint
    url="https://api.geoapify.com/v1/geocode/search?text=" + address + "&apiKey=1f61972fda3446ec9749666fb40ad436"


    #The response variable
    response = requests.get(url)

    #Get the response
    #response.text

    #Convert the response to JSON format
    data = response.json()

    #Get the lats and lons
    lat = data['features'][0]['properties']['lat']
    lon = data['features'][0]['properties']['lon']

    return lat, lon

In [52]:
address_sample['lat/lon'] = address_sample['converted_address'].apply(lambda x: get_lat_lon(x))

In [53]:
address_sample

,Patient Address,Patient Address Line 1,Patient City,Patient ZIP,Patient State,converted_address,lat/lon
317598,"15403 40th Drive SE Bothell, WA 98012",15403 40th Drive SE,Bothell,98012,WA,"15403%2040th%20Drive%20SE%20Bothell,%20WA%2098012","(47.780861, -122.179237)"
184505,"5124 322nd Ave Se Fall City, WA 98024",5124 322nd Ave Se,Fall City,98024,WA,"5124%20322nd%20Ave%20Se%20Fall%20City,%20WA%20...","(47.554222, -121.910894)"
23978,"350 Chrisman Oaks Trl Nicholasville, KY 40356",350 Chrisman Oaks Trl,Nicholasville,40356,KY,"350%20Chrisman%20Oaks%20Trl%20Nicholasville,%2...","(37.8806341, -84.5729961)"
273629,"11610 Hunter Lane NW Gig Harbor, WA 98332",11610 Hunter Lane NW,Gig Harbor,98332,WA,"11610%20Hunter%20Lane%20NW%20Gig%20Harbor,%20W...","(47.363827, -122.602205)"
223024,"7605 Snowberry Ave SE Snoqualmie, WA 98065",7605 Snowberry Ave SE,Snoqualmie,98065,WA,"7605%20Snowberry%20Ave%20SE%20Snoqualmie,%20WA...","(47.532672124485394, -121.85978640248848)"
306206,"14322 Holly Burn Ln NW Gig Harbor, WA 98329",14322 Holly Burn Ln NW,Gig Harbor,98329,WA,"14322%20Holly%20Burn%20Ln%20NW%20Gig%20Harbor,...","(47.3259697, -122.5878659)"
343132,"18016 29th Dr SE Mill Creek, WA 98012",18016 29th Dr SE,Mill Creek,98012,WA,"18016%20%2029th%20Dr%20SE%20Mill%20Creek,%20WA...","(47.83401172048308, -122.19327852111473)"
141400,"3375 Morningstar Dr Bellingham, WA 98226",3375 Morningstar Dr,Bellingham,98226,WA,"3375%20Morningstar%20Dr%20Bellingham,%20WA%209...","(48.7544012, -122.478836)"
424658,"Po Box 1801 Buckley, WA 98321",Po Box 1801,Buckley,98321,WA,"Po%20Box%201801%20Buckley,%20WA%2098321","(47.1622483, -122.027746)"
293877,"13241 SE 43rd Street Bellevue, WA 98006",13241 SE 43rd Street,Bellevue,98006,WA,"13241%20SE%2043rd%20Street%20Bellevue,%20WA%20...","(47.56747528571428, -122.16328364285715)"


In [63]:
#Export as a CSV
#care_centers.to_csv('nominatim_care_center_lat_lons.csv', index=False)